In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key='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'
    # other params...
)
response = llm.invoke("what is a vector ?")
print(response.content);


**Definition:** A vector is a mathematical object that has both magnitude (length) and direction. It is often represented graphically as an arrow in a coordinate system, with the length of the arrow representing the magnitude and the direction of the arrow representing the direction.

**Key Characteristics:**

1. **Magnitude**: The length or size of the vector, often denoted by the symbol || (e.g., ||v||).
2. **Direction**: The orientation of the vector in space, often represented by the arrowhead.
3. **Components**: Vectors can be broken down into their component parts, which are the projections of the vector onto the coordinate axes (e.g., x, y, z).

**Types of Vectors:**

1. **Zero Vector** (0): A vector with zero magnitude and no direction.
2. **Unit Vector**: A vector with a magnitude of 1 and a specific direction.
3. **Position Vector**: A vector that represents the position of a point in space relative to the origin.
4. **Displacement Vector**: A vector that represents the chang

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.ea.com/en_US/careers/JobDetail/Game-Product-Intern/210938")
page_data = loader.load().pop().page_content
print(page_data)


















        Game Product Intern - 210938 - Electronic Arts
    












                        Skip to content                    





Electronic Arts












                Open Roles
            





                About EA
            





                EA Careers
            





                EA News
            







                Login
            







                                English - US
                            


ChineseEnglish - USFinnishFrançaisItalianKoreanMalayPolishRomanianSpanishSwedish













        Game Product Intern
    










                                Home
                            



                                ...
                            



                                Open Roles
                            



                                Role Detail
                            

















                
                General Information
            






Locations: H

In [12]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITTE:
        {page_data}
        ### INSTRUCTIONS:
        the scrapped text is from career's page of a website.
        your job is to extract the job postings and return them into JSOM format containing following keys:
        'role','experience','skills'and'description'.
        if no experience then write "Fresher" in the expirience coloumn
        only return valid json
        ### VALID JSON (NO PREAMBLE):
    """
        )
chain_extract= prompt_extract | llm
res=chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Game Product Intern',
 'experience': 'Fresher',
 'skills': ['Strong analytical and problem-solving skills',
  'Excellent written and verbal communication skills',
  'Familiarity with product management tools and software (e.g., JIRA, Confluence)',
  'SQL: The ability to write complex SQL queries to analyze our databases',
  'Analytical coding: Using tools such as R, Python (and also Excel) for analytical purposes'],
 'description': 'Electronic Arts creates next-level entertainment experiences that inspire players and fans around the world. The Product Manager rolls into the Product Management function within EA and is a key element of the Game Atomic Unit - the core unit that builds the game. Building games involves a healthy balance between Art & Science and the Product Manager should be able to affect change through data driven decision making and collaboration within the larger team.'}

In [14]:
type(json_res)

dict

In [23]:
import pandas as pd 

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"MERN Stack (MongoDB, Express.js, React.js, Nod...",https://github.com/Toshtosty/Hoptalk
1,"React, Tailwind CSS, Vite",https://github.com/Toshtosty/unistudent
2,"PHP, SQL, JavaScript",https://github.com/Toshtosty/yum-yum-
3,"Deep Learning, MATLAB, CNN (Residual + Incepti...",https://github.com/Toshtosty/cracknet_pro
4,"Python, OpenCV, TensorFlow",https://github.com/Toshtosty/ml-experiments
5,"Generative AI, NLP, OpenAI API",https://github.com/Toshtosty/genai-cold-email-...
6,"C++, Data Structures & Algorithms",https://github.com/Toshtosty/dsa-practice
7,"Full-Stack Web Development, REST APIs",https://github.com/Toshtosty/fullstack-projects


In [24]:
import chromadb

import uuid
client = chromadb. PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")
 #if collection is empty i.e. we are making the collection for the first time then we will iterate
 #through complete csv file to store the entries one by one
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())])

In [25]:
links=collection.query(query_texts=['skills'],n_results=2).get('metadatas', [])
links

[[{'links': 'https://github.com/Toshtosty/genai-cold-email-generator'},
  {'links': 'https://github.com/Toshtosty/fullstack-genai-app'}]]

In [26]:
job=json_res
job['skills']

['Strong analytical and problem-solving skills',
 'Excellent written and verbal communication skills',
 'Familiarity with product management tools and software (e.g., JIRA, Confluence)',
 'SQL: The ability to write complex SQL queries to analyze our databases',
 'Analytical coding: Using tools such as R, Python (and also Excel) for analytical purposes']

In [27]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Toshita Garg, a final-year Computer Science undergraduate applying for jobs. 
    Write a professional cold email to the recruiter/hiring manager regarding the job mentioned above. 
    Highlight your relevant skills such as C++, DSA, MERN stack, and machine learning. 
    Refer to your portfolio links {link_list} to demonstrate projects that match the job requirements.
    Emphasize your ability to learn quickly, solve complex problems, and contribute effectively to their team.
    Keep the email concise, professional, and persuasive.

    ### EMAIL (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Application for Game Product Intern at Electronic Arts

Dear Hiring Manager,

I am excited to apply for the Game Product Intern position at Electronic Arts, as advertised. With a strong foundation in Computer Science and a passion for gaming, I am confident that my skills and experience make me an ideal fit for this role.

As a final-year undergraduate, I have developed a solid grasp of programming languages, including C++ and proficiency in Data Structures and Algorithms (DSA). Additionally, I have experience with the MERN stack and machine learning, which I believe can be leveraged to drive data-driven decision making in game development. My analytical and problem-solving skills, combined with excellent communication skills, enable me to effectively collaborate with cross-functional teams.

I would like to draw your attention to my portfolio, which showcases projects that demonstrate my capabilities in software development and data analysis. You can find my projects at https